In [1]:
from src import utils
from importlib import reload
import pandas as pd
import numpy as np
reload(utils)

<module 'src.utils' from 'C:\\Users\\Lucas\\Documents\\GitHub\\Poli\\PCS5708\\src\\utils.py'>

In [2]:
df = utils.read_and_treat_data()
chosen_vars = ['currentRatio', 'grossProfitMargin', 'debtEquityRatio', 'assetTurnover', 'operatingCashFlowPerShare']
new_df = utils.enconding_ratios(df,chosen_vars)
val_df = pd.DataFrame()

In [3]:
val_idxs = [238,235]+[1442,1444]+[31,32]+[248,245]+[500,498]+[1205,1206]+[369,366]+[365,367]+\
[1458,1457]+[1348,1351]+[793,792]+[1546,1544]

In [4]:
val_df = new_df.loc[val_idxs]

In [5]:
drs = {}
for t in val_df.Symbol.unique():
    try:
        r0,r1 = val_df.loc[(val_df['Symbol']==t)][['Rating','year']].sort_values(['year'])['Rating']
    except:
        r0,r1,r2,r3 = val_df.loc[(val_df['Symbol']==t)][['Rating','year']].sort_values(['year'])['Rating']
        if (r2,r3) in drs.keys():
            drs[(r2,r3)] += 1
        else:
            drs[(r2,r3)] = 1
    if (r0,r1) in drs.keys():
        drs[(r0,r1)] += 1
    else:
        drs[(r0,r1)] = 1       
drs

{('AAA', 'AAA'): 1,
 ('AA', 'AA'): 1,
 ('AA', 'A'): 1,
 ('BB', 'BB'): 3,
 ('B', 'BB'): 1,
 ('BB', 'BBB'): 1,
 ('BBB', 'BB'): 1,
 ('BBB', 'BBB'): 1,
 ('B', 'B'): 1,
 ('CCC', 'CCC'): 1}

In [6]:
df.drop(val_idxs,inplace=True)

In [7]:
val_df.to_csv('./data/validation_dataset.csv',index=False)

In [8]:
df.to_csv('./data/training_dataset.csv',index=False)

In [9]:
j=0
df = df.sort_values(['Date'])
for y in sorted(df['year'].unique()):
    df1 = df.loc[df['year']==y].copy()
    i=0
    for t in df1.Symbol.unique():
        if df.loc[(df['year']==y) & (df['Symbol']==t)].shape[0] > 1:
            i+=1
            if df.loc[(df['year']==y-1) & (df['Symbol']==t)].shape[0] == 0:
                df.loc[(df['year']==y) & (df['Symbol']==t), 'year'] = [y-1,y]
            elif df.loc[(df['year']==y+1) & (df['Symbol']==t)].shape[0] == 0:
                df.loc[(df['year']==y) & (df['Symbol']==t), 'year'] = [y,y+1]
            else:
                j+=1
                idx1, idx2 = df.loc[(df['year']==y) & (df['Symbol']==t)].index
                df.drop([idx2], inplace=True)
    if i > 0:
        print(f'Number of repeated ticker in same year for year {y}: {i}')
print(f'Number of dropped rows: {j}')

Number of dropped rows: 0


In [10]:
silence=False

In [11]:
total = 0
TICKERS_YEARS = {y:[] for y in df['year'].unique()}
for y in sorted(df['year'].unique()):
    df1 = df.loc[df['year']==y].copy()
    i=0
    for t in df1.Symbol.unique():
        if not df.loc[(df['year']==y+1) & (df['Symbol']==t)].empty:
            i+=1
            TICKERS_YEARS[y].append(t)
    if i>0 and not silence:
        print(f'Numero de tickers sendo avaliados seguidamente entre os anos {y} e {y+1}: {i}')
    total+=i
if not silence:
    print(f'Numero total de tickers sendo avaliados seguidamente: {total}')

Numero de tickers sendo avaliados seguidamente entre os anos 2010 e 2011: 14
Numero de tickers sendo avaliados seguidamente entre os anos 2011 e 2012: 106
Numero de tickers sendo avaliados seguidamente entre os anos 2012 e 2013: 208
Numero de tickers sendo avaliados seguidamente entre os anos 2013 e 2014: 244
Numero de tickers sendo avaliados seguidamente entre os anos 2014 e 2015: 316
Numero de tickers sendo avaliados seguidamente entre os anos 2015 e 2016: 282
Numero de tickers sendo avaliados seguidamente entre os anos 2016 e 2017: 38
Numero total de tickers sendo avaliados seguidamente: 1208
